# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/neSFl/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/dsECZ/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298150722802                   -0.85    5.1         
  2   -8.300169104317       -2.69       -1.25    1.2    105ms
  3   -8.300441005139       -3.57       -1.89    2.1    163ms
  4   -8.300461224662       -4.69       -2.76    2.6    126ms
  5   -8.300464034794       -5.55       -3.06    2.9    141ms
  6   -8.300464328916       -6.53       -3.22   10.4    222ms
  7   -8.300464487255       -6.80       -3.36    1.8    120ms
  8   -8.300464572736       -7.07       -3.50    2.8    155ms
  9   -8.300464628968       -7.25       -3.72    2.6    146ms
 10   -8.300464636919       -8.10       -3.86    1.1    114ms
 11   -8.300464643614       -8.17       -4.18    1.6    129ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.64175547573                   -0.70    5.4         
  2   -16.64599017615       -2.37       -1.14    1.6    253ms
  3   -16.67919412090       -1.48       -1.87    2.0    311ms
  4   -16.67927932455       -4.07       -2.66    4.2    341ms
  5   -16.67928598587       -5.18       -2.99    5.5    410ms
  6   -16.67928618616       -6.70       -3.30    2.5    282ms
  7   -16.67928621589       -7.53       -3.77    1.8    261ms
  8   -16.67928622123       -8.27       -4.50    2.8    317ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32798954906                   -0.56    7.1         
  2   -33.33515285439       -2.14       -1.00    1.6    1.03s
  3   -33.33675626908       -2.79       -1.71    8.6    1.51s
  4   -33.33691747191       -3.79       -2.63    3.1    1.08s
  5   -33.33694301756       -4.59       -3.10    4.8    1.43s
  6   -33.33694370239       -6.16       -3.94    6.2    1.34s
  7   -33.33694378173       -7.10       -4.41    4.9    1.52s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298098427907                   -0.85    5.5         
  2   -8.300250839586       -2.67       -1.58    1.2   85.3ms
  3   -8.300433115476       -3.74       -2.56    2.4    101ms
  4   -8.300387339086   +   -4.34       -2.31    3.6    170ms
  5   -8.300464000412       -4.12       -3.39    1.0   80.1ms
  6   -8.300464484699       -6.31       -3.68    2.5    125ms
  7   -8.300464639585       -6.81       -4.30    1.8    101ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32498155212                   -0.56    7.4         
  2   -33.27309580042   +   -1.28       -1.25    1.0    1.17s
  3   +14.28434034096   +    1.68       -0.23    8.1    2.07s
  4   -33.29767264733        1.68       -1.60    6.6    1.92s
  5   -33.21327512765   +   -1.07       -1.31    3.9    1.34s
  6   -32.79218245553   +   -0.38       -1.19    3.8    1.29s
  7   -33.24108495134       -0.35       -1.55    4.0    1.23s
  8   -33.33450832282       -1.03       -2.23    2.6    995ms
  9   -33.33616544465       -2.78       -2.32    3.2    1.30s
 10   -33.33682174007       -3.18       -2.65    2.2    1.01s
 11   -33.33692578969       -3.98       -2.95    2.1    986ms
 12   -33.33693289579       -5.15      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.